In [87]:
from bs4 import BeautifulSoup as BS
import pandas as pd

In [2]:
from src.config import data_dir

In [3]:
filename = "message.html"

In [4]:
with (data_dir / filename).open() as f:
    soup = BS(f, "lxml")

In [64]:
INOUT_MAP = {"main-left": "inbox",
             "main-right": "outbox"}

In [88]:
contacts = soup.find_all("div", class_="main")

In [89]:
data = []
for el in contacts[:-1]:
    record = {}
    contact_tag = el.find("div", class_="main-title")
    contact_name = contact_tag.string
    phone = el.find_next("p").get_text().split("Phone:")[-1].strip()
    record["name"] = contact_name
    record["phone"] = phone
    for p in el.find_all("p")[1:]:
        date = p.string.split("Date:")[-1].strip()
        msg = p.find_next("div")
        type_ = INOUT_MAP.get(msg["class"][0])
        text = msg.find_next("td", class_="mid-c").string.strip()
        record["date"] = date
        record["type"] = type_
        record["text"] = text
        data.append(record.copy())

In [95]:
pd.DataFrame(data).to_excel(data_dir / "tanya-sms-raw.xlsx", index=False)